<a href="https://colab.research.google.com/github/ogulcangok/foodstyles/blob/main/FoodStyles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import re
import json
!pip install spacy==3.3.0
!pip install spacy[transformers]

/usr/bin/python3: No module named spcay


In [3]:
!python -m spacy download  en_core_web_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.3.0/en_core_web_lg-3.3.0-py3-none-any.whl (400.7 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
df = pd.read_csv("/content/drive/MyDrive/recipes.csv",sep=";")

In [5]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.4)
train_json=[]

In [6]:
tj = train.to_dict(orient='records')
for recipe in tj:
  tmp = {"entities":[],
         "text":recipe["Directions"]}
  for ingredient in recipe["Ingredients"].split(","):
    try:
      match = re.search(ingredient,recipe["Directions"])
      if match:
          tmp["entities"].append(("ingredient",match.start(), match.end()))
          train_json.append(tmp)
      else:
          pass
    except TypeError:
      pass

In [7]:
train_json[0]

{'entities': [('ingredient', 56, 62),
  ('ingredient', 77, 88),
  ('ingredient', 90, 101),
  ('ingredient', 222, 227)],
 'text': 'Preheat oven to 350 degrees F (180 degrees C).**Combine butter or margarine, baking soda, white sugar and your favorite extract flavor.  Beat these ingredients together with an electric mixer until fluffy. Add self-rising flour  Beat well.**Roll dough into walnut sized balls. Place them two inches apart onto the cookie sheet and flatten slightly. Press ice cream stick into the flattened side of the cookie, and press down. This way, you know how large the cookie will be when baked. Add whatever topping to each cookie you like, OR, before pressing stick into the cookie, roll each cookie into sprinkles first, then flatten.**Bake until edges are lightly golden.**'}

In [9]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import random
nlp = spacy.load("en_core_web_lg") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [10]:
from spacy.util import filter_spans

for training_example  in tqdm(train_json): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for label, start, end in labels:
        span = doc.char_span(start, end, label=label,alignment_mode="contract")
        if span is None:
            pass
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

100%|██████████| 49214/49214 [00:30<00:00, 1598.03it/s]


In [11]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-05-21 09:32:44,991] [INFO] Set up nlp object from config
[2022-05-21 09:32:45,003] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-21 09:32:45,008] [INFO] Created vocabulary
[2022-05-21 09:32:47,330] [INFO] Added vectors: en_core_web_lg
[2022-05-21 09:32:48,725] [INFO] Finished initializing nlp object
tcmalloc: large alloc 1073750016 bytes == 0x6ad52000 @  0x7f5bb310c2a4 0x592b76 0x4286f3 0x5bb021 0x5bb176 0x5936cc 0x548c51 0x51566f 0x593dd7 0x511e2c 0x593dd7 0x511e2c 0x598ef4 0x511f68 0x598ef4 0x511f68 0x598ef4 0x7f5a63b558a6 0x4bc4ab 0x7f5a63b53dc3 0x593784 0x515244 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x5127f1
[2022-05-21 09:33:55,686] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']


In [13]:
nlp_ner = spacy.load("model-best")

In [14]:
testj = test.to_dict(orient='records')

In [15]:
doc = nlp_ner(testj[0]["Directions"])
spacy.displacy.render(doc, style="ent", jupyter=True)

In [23]:
print([(ent.text, ent.start,ent.end) for ent in doc.ents])

[('cinnamon', 27, 28), ('vanilla', 29, 30)]


In [19]:
from google.colab import files
files.download('/content/model-best') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
def get_ingredients(model,recipes):
  nlp_ner = spacy.load(model)
  result = {}
  for i,text in recipes.items():
    doc = nlp_ner(text)
    result[i] = [(ent.text, ent.start,ent.end) for ent in doc.ents]
  
  return result

In [22]:
fn_test ={
    "recipe1":testj[0]["Directions"],
    "recipe2":testj[1]["Directions"]
}
get_ingredients("model-best",fn_test)

{'recipe1': [('cinnamon', 27, 28), ('vanilla', 29, 30)],
 'recipe2': [('turkey', 2, 3)]}